In [8]:
import cv2
import json
# Open the video file
video_path = 'DeepSquat1 (1).avi'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Cannot open camera")

# Define the keypoint mapping for this OpenPose body_25 model
keypoints_mapping = {
    0:  "Nose", 1:  "Neck", 2:  "RShoulder", 3:  "RElbow", 4:  "RWrist", 5:  "LShoulder", 6:  "LElbow",
    7:  "LWrist", 8:  "MidHip", 9:  "RHip", 10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee",
    14: "LAnkle", 15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe", 20: "LSmallToe",
    21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel", 25: "Background"
}

humanML_mapping = {
    0: "root", 1: "RH", 2: "LH", 3: "BP", 4: "RK", 5: "LK", 6: "BT", 
    7: "RMrot", 8: "LMrot", 9: "BLN", 10: "RF", 11: "LF", 12: "BMN", 13: "RSI", 
    14: "LSI", 15: "BUN", 16: "RS", 17: "LS", 18: "RE", 19: "LE", 20: "RW", 21: "LW"
}

#compatible_mapping = [12, 9, 16, 18, 20, 17, 19, 21, 0, 1, 4, x, 2, 5, x, X, X, X, X, X, X, 8, x, x, 7, x]

humanml_from_openpose = [8, 4, 7, 8, 10, 13, 8, 24, 21, 1, 22, 19, 0, 2, 5, 0, 2, 5, 3, 6, 4, 7] #check BMN, hand 

humanmlframes = []

#load the model
net = cv2.dnn.readNetFromCaffe('pose_deploy.prototxt', 'pose_iter_584000.caffemodel')

def framestep(cap):
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    print("fps: %d" %(fps))
    print()
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    new_vid = []
    if (fps % 10 == 0): #we can easily do 10 fps
        stepsize = fps/1 # reset to 10
        print("total frames to be analyzed: ", frame_count / stepsize)
        curr_frame = 0
        i = 0
        while curr_frame < frame_count:
            ret, frame = cap.read()
            if curr_frame % stepsize == 0:
                print("starting frame: ", i)
                new_vid.append(pose(frame))
                i+=1
            curr_frame+=1
    
    else:
        print("NOT BUILT YET")
        pass
    
    return new_vid
    
def squarify(frame):
    height, width, _ = frame.shape
    min_dim = min(height, width)

    # Calculate the cropping dimensions
    crop_height = (height - min_dim) // 2
    crop_width = (width - min_dim) // 2

    # Crop the image equally from both sides to make it a square
    return frame[crop_height:crop_height+min_dim, crop_width:crop_width+min_dim] , min_dim

def pose(frame):
    
    frame, size = squarify(frame)
    blob = cv2.dnn.blobFromImage(frame, 1/255, (size, size),
                             (0, 0, 0), swapRB=False, crop=True)

    # run forward pass to get the pose estimation
    net.setInput(blob)
    output = net.forward()
    
    # Extract joint locations
    joint_locations = []

    for i in range(len(keypoints_mapping) - 1): #-1 bc we dont want point for the background
        keypoint = output[0, i, :, :]
        min_val, confidence, min_loc, point = cv2.minMaxLoc(keypoint)

        if confidence > 0.1:  # can adjust the confidence threshold if needed ???
            joint_locations.append((8 * int(point[0]), 8 * int(point[1]), i))
        else:
            joint_locations.append(None)

    #joint_locations contains the locations of the detected joints and corresponding index
    pose_data = {}
    for location in joint_locations:
        if location:
            x, y, index = location
            joint_name = keypoints_mapping[index]
            pose_data[joint_name] = {'x': x, 'y': y}

    return pose_data

def process_for_motiongpt(new_vid):
    formatted_data = json.dumps(new_vid)
    return formatted_data

# Process the video frames
my_video = framestep(cap)

# Prepare the data for MotionGPT
formatted_motion_data = process_for_motiongpt(my_video)

# Assuming you have a function to feed data into MotionGPT and get results
motiongpt_results = motiongpt_process(formatted_motion_data)

cap.release()

Cannot open camera


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\caffe\caffe_io.cpp:1138: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "pose_iter_584000.caffemodel" in function 'cv::dnn::ReadProtoFromBinaryFile'
